## Kaggle Lux AI Competition

This is my first ever Simulation competition on Kaggle. I actually was quite lost when I started, but then I picked up the starter [notebook](https://www.kaggle.com/ilialar/lux-ai-risk-averse-baseline) from Ilia Larchenko to try and understand what was happening and then began writing my own code from scratch. After spending several days between writing code and debugging, I have a somewhat intelligent rule based agent that's currently scoring 600+ points on leaderboard. 

These were some of the strategies deployed on my bot - 
* **Deferring City Growth** - As a defensive tactic, I deferred growth of cities until sufficient fuel was collected. I defined different thresholds for fuel collection based on the size of the map. Until the threshold was hit I didn't add a second tile on the map. This was done to ensure that the first city has enough fuel to survive rest of the game before further tiles and cities are added. This was done primarily to ensure that in the case all resources and units on the maps are exhausted, there is atleast one city surviving the game
* **Identifying tiles for City growth** - The amount of fuel required per city tile is inversely related to the number of adjacent city tiles. Therefore in order to give our cities better chance of survival, it is important to ensure that new tiles are either added closer to resources or adjacent to already existing citytiles. For every unit, I try to find the nearest adjacent city tile and move the unit towards it for constructing a new city tile. 
* **Collision Handling** - In my initial bots, lots of moves were getting cancelled because units ended up colliding with each other. I had to write a collision handling logic where I maintain a list with tiles occupied by all units and before moving a particular unit, I run a check to see if the tile where a unit is looking to move is unoccupied. This list keeps getting updated after every turn to ensure the latest position of all tiles is getting captured. 
* **Surviving the night** - If the cities & units don't have enough fuel before the onset of night, then there is a good chance that they may end up dying. Therefore its very important to ensure that all units & cities are stocked with enough fuel to survive the night. I applied checks on game turns to see if a night was approaching and ensured that units started moving back to cities before the onset of the night. Cities were sorted in the order of lowest volume of fuel and units were then assigned to cities. When night started approaching, every unit started moving back to the city assigned to it
* **Cluster Identification** - An important aspect of this game is to ensure that the bot is able to identify big cluster of resources away from the starting point and starts growing cities there. This strategy kicks in if there are sufficient number of resources on the map. The last unit starts moving towards the biggest available cluster and starts growing the city there.
* **Research** - Lots of maps available on the game environment have plenty of coal and uranium resources available. A city tile on any given turn has the option to either produce a worker or conduct research. The idea in initial turns is to focus more on research so that all the coal resources become accessible. Research is stopped when the uranium research is complete and thereafter the focus is only on producing more workers. 
* **Exception Handling** - I noticed a lot of matches were lost by my initial bots due to exception conditions like all resources getting exhausted on the maps or all city tiles dying. I added several try catch blocks to ensure I don't end up losing the match midway through the game. 


This is the simple strategy deployed on my bot so far - 
1. At the beginning of the game, start moving towards resource tiles & collect resources
2. Until a particular growth threshold has been reached, continue depositing the resources to the initial city tile
3. Once the growth threshold has been reached, identify empty tiles adjacent to the citytile and start moving there to construct a new city tile
4. If a night is approaching, ensure the unit moves back to the city assigned to it
5. Once there are enough number of units on the map, ensure that the last unit is always looking to move towards the biggest cluster on the map. Once it gets there it should start growing a city on that cluster

In [ ]:
!pip install kaggle-environments -U

In [ ]:
from kaggle_environments import make
import json

In [ ]:
# create the environment. You can also specify configurations for seed and loglevel as shown below. If not specified, a random seed is chosen. 
# loglevel default is 0. 
# 1 is for errors, 2 is for match warnings such as units colliding, invalid commands (recommended)
# 3 for info level, and 4 for everything (not recommended)
# set annotations True so annotation commands are drawn on visualizer
# set debug to True so print statements get shown
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
# run a match between two agents, simple_agent is a default one
steps = env.run(["simple_agent", "simple_agent"])

In [ ]:
# you can also render the replay inline as so. we recommend for best viewing experience to set width and height as large possible
# if you are viewing this outside of the interactive jupyter notebook / kaggle notebooks mode, this may look cutoff
# you may also want to close the output of this render cell or else the notebook might get laggy
#env.render(mode="ipython", width=500, height=500)


In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .
# if working locally, download the `simple/lux` folder from here https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits/python
# and we recommend following instructions in there for local development with python bots

In [ ]:
%%writefile agent.py
# we add this above so you can write this main agent code into a file for submission later

# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import numpy as np
import math
import random
import copy

def get_random_step():
    return np.random.choice(['s','n','w','e'])

### Define helper functions
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile, closest_dist

#Finds the closest city tile to a particular player
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    closest_dist = math.inf
    if len(player.cities) > 0:        
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile, closest_dist

def find_player_city(player,unit,player_tile_mapping):
    closest_city_tile = None
    destination_city=player_tile_mapping[unit.id]
    closest_dist = math.inf
    pos=unit.pos
    if len(player.cities) > 0:
        for k,city in player.cities.items():
            print(city)
            if city.cityid==destination_city:
                 for city_tile in city.citytiles:
                    dist = city_tile.pos.distance_to(pos)
                    if dist < closest_dist:
                        closest_dist = dist
                        closest_city_tile = city_tile
                        print(unit.id,'moving to city',city.cityid)
                
    return closest_city_tile

#Find the total number of city tiles created by the player
def find_total_city_tiles(player):
    total_tiles=0
    for k, city in player.cities.items():
            for city_tile in city.citytiles:
                total_tiles+=1            
    return total_tiles

#Collision Handling logic. Checks if the proposed move is valid or not
def check_move_validity(unit,moved_pos_coord,directions):
    direction=None
  #  print('Unit',unit.id,'Proposed move',moved_pos_coord)
  #  print('Collision list',collision_list)
    moved_pos_tile=None
    
    if moved_pos_coord not in collision_list:
        direction=directions
#        collision_list.append([moved_pos_coord[0],moved_pos_coord[1]])
        moved_pos_tile=moved_pos_coord
        print("Updated")        
    else : 
        i=0
#        print('Checking rando')
        for i in range(4):
            directions=get_random_step()
            moved_pos=unit.pos.translate(directions,1)
            moved_pos_coord=[moved_pos.x,moved_pos.y]
            if moved_pos_coord not in collision_list:
                moved_pos_tile=moved_pos_coord
                print('Potential Collision detected, assigning random direction ',directions)
                break
    print('Unit',unit.id,'Accepted move',moved_pos_coord)
    return directions,moved_pos_tile

#If a move has been approved, updates the list of occupied tiles for all units of the player
def update_collision_list(unit,moved_pos_tile):
    print('updating collision list',moved_pos_tile)
    collision_list.append([moved_pos_tile[0],moved_pos_tile[1]]) 
    try:
        collision_list.remove([unit.pos.x,unit.pos.y])
    except :
        print('Collision list removal exception occured',unit.pos.x,unit.pos.y)

#Finds the adjacent tiles to all the citytiles to find new potential positions for citytiles
def find_adjacent_citytiles(player):
    x=0
    y=0
    pos_empty_pos=[]
    for k, city in player.cities.items():
        for city_tile in city.citytiles:
            x=city_tile.pos.x
            y=city_tile.pos.y
         #   print('City tile looking at ',x,y)
            try:
                dirs=[(1,0),(-1,0),(0,1),(0,-1)]
                for d in dirs:                        
                    possible_empty_tile = game_state.map.get_cell(x+d[0], y+d[1])       
                  #  print("Possible empty tile",possible_empty_tile.pos)
                    if possible_empty_tile.has_resource() or possible_empty_tile.citytile is not None:
                        continue
                    else :
                    #    print('Being appended',possible_empty_tile.pos)
                        pos_empty_pos.append(possible_empty_tile.pos)
                           #     action = unit.move(unit.pos.direction_to(pos_empty_pos))
                           #     actions.append(action)
                      #  print("Empty position found",possible_empty_tile.pos)
                        posFound=True
            except Exception as e:
                #print('Exception occured')
                k=0
         #   print(posFound,' posFound')    
    return pos_empty_pos

#Sorts the list of all potential empty tiles to find the tile closest to a unit. 
def sort_tiles(unit,pos_empty_pos):
    unit_pos=unit.pos
    dist_list=[]
    for pos in pos_empty_pos:
       # print('Print looking at tile',pos.x,pos.y)
        tile_pos=pos      
        dist = tile_pos.distance_to(unit_pos)
        dist_list.append(dist)
    #print('Dist list',dist_list)    
   # pos_empty_pos_sorted=[x for _, x in sorted(zip(dist_list, pos_empty_pos))]
    dic=dict(zip(dist_list,pos_empty_pos))
    dic=dict(sorted(dic.items()))
#    print(dic)
    pos_empty_pos_sorted=dic.values()
  #  print('sorted list is ',pos_empty_pos_sorted)
    return pos_empty_pos_sorted
        

#Identifies all the clusters of available resources on the map and then sorts them in a descending order based on size
def cluster_identification(player,game_state):
    resource_tiles=find_resources(game_state)
    cluster_size=0
    cluster_size_mapping={}
    dirs=[(1,0),(-1,0),(0,1),(0,-1),(2,0),(-2,0),(0,2),(0,-2)]
    pos_list=[]
    for tile in taken_tiles:
        pos_list.append(tile.pos)
    
    for tile in resource_tiles:
        if tile.pos in pos_list:
            continue
        
        x,y=tile.pos.x,tile.pos.y
        cluster_size=0
        for d in dirs:     
            try:
                possible_resource_tile = game_state.map.get_cell(x+d[0], y+d[1])   

            except Exception as e:
                continue
#                print('Tile overrun')
            if possible_resource_tile.has_resource():
                if possible_resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
                if possible_resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
                cluster_size=cluster_size+1
      #  print('Evaluating',tile.pos,cluster_size)
        cluster_size_mapping[tile]=cluster_size
    #    print('tile pos & cluster_size',x,y,cluster_size)    
    cluster_size_mapping=sorted(cluster_size_mapping.items(), key=lambda x: x[1], reverse=True)
    return cluster_size_mapping
   # print(cluster_size_mapping)
    
#Provides the direction a unit needs to take to move closer to a particular cluster
def move_towards_cluster(unit):
    big_tile=cluster_size_mapping[0][0]
    print('Big tile',big_tile.pos,' resources ',cluster_size_mapping[0][1])
    pos=big_tile.pos
    moved_pos=unit.pos.translate(unit.pos.direction_to(pos),1)
    moved_pos_coord=[moved_pos.x,moved_pos.y]
    directions,moved_pos_tile=check_move_validity(unit,moved_pos_coord,unit.pos.direction_to(pos))
    action = unit.move(directions)
    print('Moving towards cluster',moved_pos)
    moved=True
    return action,big_tile

# this snippet finds the number of coal and wood resources on the map
def find_resource_size(game_state):
    coal=0
    wood=0
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                if cell.resource.type == Constants.RESOURCE_TYPES.COAL:
                    coal=coal+1
                if cell.resource.type == Constants.RESOURCE_TYPES.WOOD:
                    wood=wood+1
    return wood,coal

#Maps the players to the cities. Cities with lowest fuel get mapped first. 
def player_city_mapping(player):
    x=0
    y=0
    pos_empty_pos=[]
    city_fuel_mapping={}
    city_player_mapping={}
    units=copy.deepcopy(player.units)

    for k, city in player.cities.items():
        fuel=city.fuel
        upkeep=city.get_light_upkeep()
        steps=int(fuel/upkeep)
        print(city.cityid,'city id fuel',fuel,upkeep,steps)
        city_fuel_mapping[city]=steps
        
    city_fuel_mapping=sorted(city_fuel_mapping.items(), key=lambda item: item[1])
    city_fuel_mapping=[i[0] for i in city_fuel_mapping]
    
    dis=0
#    print(city_fuel_mapping)
    for city in city_fuel_mapping:
        city_tile=city.citytiles[0]
        tile_pos = city_tile.pos
        min_dis=math.inf
        min_unit=None
        if len(units)>0:
            for unit in units:
                dis=tile_pos.distance_to(unit.pos)
                if dis<=min_dis:
                    min_dis=dis
                    min_unit=unit
            city_player_mapping[min_unit.id]=city
            units.remove(min_unit)
   # print('City unit mapping',city_player_mapping)
    return city_player_mapping
        

# we declare this global game_state object so that state persists across turns so we do not need to reinitialize it all the time
game_state = None
game_state = None
posFound=False
pos_empty_pos=[]
cluster_unit=None
big_tile=None
# this is the basic agent definition. At the moment this agent does nothing (and actually will survive for a bit before getting consumed by darkness)
def agent(observation, configuration):
      
    global game_state
    global game_map
    global taken_tiles
    global posFound
    global pos_empty_pos
    global collision_list
    global cluster_size_mapping
    global cluster_unit
    global big_tile
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
        game_map=game_state.map
    else:
        game_state._update(observation["updates"])
    
    actions = []
    action=None
    moved=False
    taken_tiles=[]
    #PlayerCityMapping
    player_tile_mapping={}
    
    #Cluster Size Mapping
    
    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    turn=game_state.turn
    width, height = game_state.map.width, game_state.map.height
    resource_tiles=find_resources(game_state)
    night=game_state.turn % 40
    isnight = 1 if night > 30 else 0
    approaching_night= 1 if night > 25 else 0
    total_city_tiles=find_total_city_tiles(player)

    cities = list(player.cities.values())
    cityfuel=0
    growth_threshold=800
    for city in cities:
        cityfuel+=city.fuel
#    print('cityfuel is'+ str(cityfuel))
    print('GAME TURN '+str(game_state.turn))    
    print('Unit volume',len(player.units))

    #Adjust growth threshold based on size of the map    
    try:
        width=game_map.width
        if width==12:
            growth_threshold=800
        elif width==16 :
            growth_threshold=1000
            
    except Exception as e:
        print('Thresholds are none')
        
    #Calculates clusters on a map every 10 turns to account for depleting resources
    if game_state.turn%10==0:
        cluster_size_mapping=cluster_identification(player,game_state)
      #  print(cluster_size_mapping)
#     for ele in cluster_size_mapping:
#         print(ele[0].pos,ele[1])
   #     raise Exception
    collision_list=[]
    for unit in player.units:
        collision_list.append([unit.pos.x,unit.pos.y])
       
    #Player City Mapping being created
    i=0
    cluster_move_flag=False
    city_player_mapping=player_city_mapping(player)

    print('City Player mapping')
    for k,v in city_player_mapping.items():
        print(k,v.cityid)
    
    for unit in player.units:
        if len(cities)>0:
            i=i%len(cities)
            player_tile_mapping[unit.id]=cities[i].cityid
            i=i+1
#    print('Player City Mapping',player_tile_mapping)
    print('Approaching night',approaching_night)
 #   print('Collision List',collision_list)   
    l=0
    id_list=[]
    for unit in player.units:
        id_list.append(unit.id)
    
    for unit in player.units:
        l=l+1
        print('UNIT',unit.id)
        pos_empty_pos=find_adjacent_citytiles(player)
        moved=False
        moved_pos_tile=None
        print('Unit acting status ',unit.can_act())
        if unit.can_act():
            closest_city_tile, closest_dist=find_closest_city_tile(unit.pos,player) 
            closest_resource_tile, closest_resource_dist = find_closest_resources(unit.pos, player, resource_tiles)
            if closest_resource_tile is None:
                continue
            
            #Moving to another cluster
            units  = len(player.units)
            
            if big_tile is not None:
                if unit.pos==big_tile.pos:
                    taken_tiles.append(big_tile)
                    cluster_unit=None
                    cluster_size_mapping=cluster_identification(player,game_state)
                
            
            if cluster_unit is not None :
                if cluster_unit.id==unit.id:
                    print('Cluster Action')
                    action,big_tile=move_towards_cluster(unit)
                    actions.append(action)
                    cluster_move_flag=True
    #                print('Moving_towards cl')
                    continue
                
            if units>=3 and (l==units) and cluster_move_flag==False :
                if cluster_unit is None or cluster_unit.id not in id_list :                    
                    print("Assigning a new cluster unit")
                    cluster_unit=unit
                    action,big_tile=move_towards_cluster(unit)
                    actions.append(action)
                    cluster_move_flag=True
                    continue
                    
            print(str(unit.id) + ' Unit has resources '+str(unit.cargo.wood))
            
    #        print(unit,' can build ',unit.can_build(game_map))
            print('unit cooldown',unit.cooldown)
            if unit.can_build(game_map):
                print('can build')
                player_tile= game_state.map.get_cell(unit.pos.x, unit.pos.y)

                if (closest_dist<=1 or closest_resource_dist==1 ): 
                    action = unit.build_city()
                    actions.append(action)
                    posFound=False
                    continue                    
            else :
                closest_resource_tile, closest_resource_dist = find_closest_resources(unit.pos, player, resource_tiles)
               # print('total city tiles'+str(total_city_tiles))
                
                if closest_dist==0 and closest_resource_dist==1 and total_city_tiles>=5:
                    continue
                    
                resource_pos=closest_resource_tile.pos
                directions = unit.pos.direction_to(resource_pos)
                
                print('closest_resource_dist is '+str(closest_resource_dist))
                if closest_resource_dist!=0:
                    print(str(unit.id) +'Unit moving to resources')
                    moved_pos=unit.pos.translate(directions,1)
                    moved_pos_coord=[moved_pos.x,moved_pos.y]
                    directions,moved_pos_tile=check_move_validity(unit,moved_pos_coord,directions)
                    print("Check",directions,moved_pos_tile)
                    action=unit.move(directions)  
                    
               #If a growth threshold has been captured then look to grow city beyond a single tile
                if cityfuel>=growth_threshold and approaching_night==False:
                    #Find all empty city tiles 
                   # pos_empty_pos=find_adjacent_citytiles(player)
                    if len(pos_empty_pos) >0 and (unit.cargo.wood+unit.cargo.coal + unit.cargo.uranium>=100):  
                        dirs_considered=[]
                      #  print('Possible Empty Tiles list',len(pos_empty_pos))
                     #   print('Possible Empty tile',pos_empty_pos[0])
                        
                        #Sort possible empty tiles list in order of distance from the unit
                        pos_empty_pos=sort_tiles(unit,pos_empty_pos)
                        #loop through all empty position tiles to see if unit can move to an empty tile for construction
                        for pos in pos_empty_pos:
                            #Unit should move to closest adjacent position
                            moved=False
                            dir=unit.pos.direction_to(pos)
                            dirs_considered.append(dir)
                            possible_empty_pos=unit.pos.translate(dir,1)
                            possible_empty_tile = game_state.map.get_cell(possible_empty_pos.x,possible_empty_pos.y)                                
                            print('Moving tile & potential tile',pos.x,pos.y,possible_empty_pos.x,possible_empty_pos.y)
                       #     print(possible_empty_tile.citytile)
                            if  (possible_empty_tile.citytile is not None):
                                print('move will lead to an occupied tile')
                                continue
                            else :
                                moved_pos=unit.pos.translate(unit.pos.direction_to(pos),1)
                                moved_pos_coord=[moved_pos.x,moved_pos.y]
                                directions,moved_pos_tile=check_move_validity(unit,moved_pos_coord,unit.pos.direction_to(pos))
                                action = unit.move(directions)
                                
                       #         collision_list.append([moved_pos.x,moved_pos.y])
                                print('moving unit to an empty tile for construction')
                                posFound=False
                                moved=True
                                pos_empty_pos=[]
                                player_tile_mapping[unit.id]=[pos.x,pos.y]
                             #   print(player_tile_mapping[unit.id])
                                break
                    
                        print('Moved ',moved)
                        #If a unit hasn't moved to an empty tile for construction then move to a random position
                        if moved==False:
                            print('Moving to a random position to find a way out')
                            directions=get_random_step()
                            moved_pos=unit.pos.translate(directions,1)
                            moved_pos_coord=[moved_pos.x,moved_pos.y]
                            directions,moved_pos_tile=check_move_validity(unit,moved_pos_coord,directions)
                            action =unit.move(directions)
                            actions.append(action)
                            continue
                            
                     #   continue               
             #   if posFound==False: 
                #If a unit has collected 100 resources or if night is approaching then move towards city
                print('moved is',moved)
                if ((unit.cargo.wood + unit.cargo.coal + unit.cargo.uranium >=100) or approaching_night==1) and moved==False:
#                    closest_city_tile, closest_dist=find_closest_city_tile(unit.pos,player)
                    #closest_city_tile=find_player_city(player,unit,player_tile_mapping)
    
                    #Tryin new approach by mappin city to closest resources. Cities are mapped in order of lowest resrouces available
                   # print('City player mapping',city_player_mapping)
                    if unit.id in city_player_mapping:
                     #   print('matched')
                        city=city_player_mapping[unit.id]
                        closest_city_tile=city.citytiles[0]
                    else:
                        closest_city_tile,closest_dist=find_closest_city_tile(unit.pos,player)
                    
                  #  print('CLosest city tile',closest_city_tile)
                    city_pos=closest_city_tile.pos
                    directions = unit.pos.direction_to(city_pos)
                    moved_pos=unit.pos.translate(directions,1)
                    moved_pos_coord=[moved_pos.x,moved_pos.y]
                    directions,moved_pos_tile=check_move_validity(unit,moved_pos_coord,directions)
                    print('before moving back to city',directions,moved_pos_tile)
                    action=unit.move(directions)        
                    print(str(unit.id) + 'moving back to city',player_tile_mapping[unit.id],'distance',unit.pos.distance_to(city_pos))

               # pos_empty_pos = list(set(pos_empty_pos))
                print('ACTION',action)
                if action is not None:
                    if moved_pos_tile is not None:
                        update_collision_list(unit,moved_pos_tile)
                    actions.append(action)              
                    
  # max number of units available
    units_cap = sum([len(x.citytiles) for x in player.cities.values()])
    # current number of units
    units  = len(player.units)
    created_worker=None         
    cities = list(player.cities.values())
 #   print('Wood Coal',wood,coal)
    if len(cities) > 0:
    #    city = cities[0]
        if units<=5:
            created_worker = (units >= units_cap)
        else :
            created_worker = (units >= int(0.7*units_cap))

        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                if city_tile.can_act():
                    if created_worker==True and player.research_points<=200:
                        # let's do research
                        action = city_tile.research()
                        print('City doing research')
                        actions.append(action)
                    else:
                        # let's create one more unit in the last created city tile if we can
                        action = city_tile.build_worker()
                        print('City creating worker')
                        actions.append(action)                    
                        created_worker = True
                #directions = []
    actions = list(set(actions))
    print('Actions ',actions)

    return actions

In [ ]:
# run another match but with our empty agent
#77935795
env = make("lux_ai_2021", configuration={"seed": 163572896, "loglevel": 2}, debug=True)
steps = env.run(['simple_agent',"./agent.py"])
env.render(mode="ipython", width=800, height=900)

In [ ]:
# save the replay as a file. if working locally this should appear in your current directory, otherwise you can download
# from this kaggle notebook by opening the output section of the data panel on the right
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)
# this replay can then be watched here https://2021vis.lux-ai.org/

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [ ]:
!tar -czf submission.tar.gz *

## Learnings and Next Steps

* **Python for Software Engineering** - As a data analyst, my usage for python so far was largely restricted to using pandas, numpy, ML packages & visualization libraries like Plotly & Seaborn. I hadn't really ever used python for a software engineering task. This competition has been a great learning for me so far where I have had to focus a lot on software engineering & debugging side of things while extensively using inbuilt data structures
* **Rapid City Growth** - More successful bots in the 1000+ range are able to completely dominate the map and grow very rapidly to 20+tiles & workers. I need to figure out a way to emulate that as well
* **Optimizing City Shape** - Cities with tiles in straight lines tend to burn more energy than cities in a polygon shape. So far my bot has a huge weakness that its developing small cities with 2-3 tiles and I am in no way optimizing for shape. Would like to do that soon.
* **Usage of Carts** - I have come up against lots of opponents using carts & roads. Its worth a try since carts can hold up a lot of fuel and move much faster than workers. 
* **Observing Opponent Behavior** - So far my bot has only been focused on its own actions and strategy for growing on the map. It hasn't been observing what an opponent is doing and changing its strategy to best deal with it. I am not yet sure how to go about this, but I feel this change would be critical to ensure I move up on the leaderboard. 
* **Reinforcement Learning** - I have been learning the RL specialization on Coursera for a month now and the primary motivation for picking up this problem was to be able to use RL for a simulation competition. I wanted to get started with a basic rule based agent to get familiar with the problem statement and the API. Now that I have been able to achieve that, the idea is to be able to hopefully create a half decent RL agent by the end of this competition. 


I will be making lots of updates to this notebook in the coming days and weeks. Please suggest improvements or provide feedback in the comments. 